In [ ]:
# IMPORTS 
import cv2
import os

# IDEE

Ausgangssituation: 
Twitter datensatz, Bilder sind als links zur Verfügung können aber runtergeladen werden 

=> Es sollen nur Bilder rausgefiltert werden, die tatsächlich Waffen in gewaltätigen Kontext zeigen. Bilder auf denen Waffen nicht eingestezt werden, können drin bleiben um die Representation von Solchen Bildmaterial zu gewährleisten. 
Programmierspraceh: Python 
-> zwei Schritte: 
1. Alle Bilder mit Waffen rausfiltern 
    * YOLOv8, 
2. Dann mit Pose-Estimation entscheiden auf welchen Bildern Aggresives Handlen zu erkennen ist 
    * je nach größe der Ergebnissemeng, Bilder manuel aussortieren
    * OpenPose 

# Erste Version zur Umsetzung 

### Schritt 1: Bilder mit Waffen erkennen 
1. Laden der YOLOv8-Modelldateien:
    * Yolov8 kann man mit Pytorch oder der ultralytics Bibliotehl verwenden 
2. Bilder asu dem Dataset herunteralden:
    Die Bild-Links aus dem Twitter-Datensatz werden heruntergeladen un dlokal gespeichert 
3. Waffen erkennen:
    * YOLOv8 wird verwendet, um Bilder zu klassifizieren, die Waffen enthalten 

### Schritt 2:  Erkennung aggresiver Handlungen 
1. Pose-Estimation mit Open-Pose:
    * OpenPose wird angewandt, um die Körperhaltung und Bewegungen der Personen im Bild erkennen 
    * Erkennungsmerkmale für aggresuve Handlungen werden analysiert, z.B. Gesten die auf Gewalt hindeuten 
2. Filtern von Ergebnissen:
    * Bilder, die als aggressiv eingestuft werden, werden separiert.
3. Optionale manuelle Nachprüfung: 
    * Je nach Größe der Ergebnisdatenmenge können die Bilder visuell überprüft werden.

In [ ]:
import os 
import requests
from ultralytics import YOLO
import cv2
import numpy as np 

In [ ]:
# Ordner für Bilder 
IMAGE_DIR = "./images"
os.makedirs(IMAGE_DIR, exist_ok=True)



In [ ]:
# 1. Bilder herunterladen 
def download_images(image_links):
    downloaded_images = []
    for i, link in enumerate(image_links):
        try:
            response = requests.get(link, stream=True)
            if response.status_code == 200:
                image_path = os.path.join(IMAGE_DIR, f"image_{i}.jpg")
                with open(image_path, 'wb') as f:
                    f.write(response.content)
                downloaded_images.append(image_path)
        except Exception as e:
            print(f"Error downloading {link}: {e}")
    return downloaded_images

In [ ]:
# 2. YOLOv8: Erkennung von Waffen
def detect_weapons(image_paths, model_path="yolov8n.pt"):
    model = YOLO(model_path)  # YOLOv8-Modell laden
    weapon_images = []
    
    for image_path in image_paths:
        results = model(image_path)
        # Prüfen, ob eine Waffe im Bild erkannt wurde
        if any(detection['name'] == 'weapon' for detection in results[0].boxes.data):
            # detection results of images analyzed 
            # resulst[0].boxes.data cinatins all the bounding boxes deteced in the image, along with metadata such as the class name (name), condfidence score, and coordinates
            # the function checks if any deteced object in the image has the class name 'weapon'. This indicates the presence of a weapon in the image 
            weapon_images.append(image_path)
    return weapon_images

1. Object Detection Capabilities:
    * is a state-of-the_art object detection algorithm optimized for detecting objects in real-time with high accuracy 
    * its architecture uses convolutional neurla networks (CNN) to identify patterns in images and classify objetcs baseed on those patterns 
    * is preptrained on diverese datasets, including  images with weapons, makng it suitabe for this task
2. Efficient Processing:
    * The grid-based detection systems in YOLO ensures that the entire image is analyzed efficiently in a single forward pass, making it fast and scalable for multiple images 
3. Selective Filtering: 
    * The function filters results based on the class name weapon. This ensures that only relevnat images are falgged, reducing the noise in subsequent steps 
4. Flexibility: 
    * By using file paths, the function is modular and can easily be adapated for different datasets or integrated into larger piplines 
    * the model_path argument allows switching between differnet YOLOv8 models ()

In [ ]:
# Hilfsfunktion zur Erkennung aggressiver Posen
def is_aggressive_pose(pose_data):
    # Placeholder für Logik zur Aggressions-Erkennung
    # z.B. erhobene Hände/Waffen, gebeugte Haltungen etc. ===> Open Pose Datei
    return True

In [ ]:
# 3. OpenPose: Erkennung aggressiver Posen
def analyze_aggressive_poses(image_paths, openpose_model_path="./openpose/models/"):
    aggressive_images = []
    # OpenPose initialisieren
    params = {"model_folder": openpose_model_path}
    openpose = cv2.dnn.readNetFromCaffe(params["prototxt"], params["caffemodel"])

    for image_path in image_paths:
        image = cv2.imread(image_path)
        h, w, _ = image.shape
        blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (368, 368), (0, 0, 0), swapRB=False, crop=False)
        openpose.setInput(blob)
        output = openpose.forward()

        # Analyse der Pose-Daten (z. B. erkennbare Aggression)
        if is_aggressive_pose(output):
            aggressive_images.append(image_path)
    return aggressive_images

In [ ]:
# Hauptfunktion
def main(image_links):
    print("1. Bilder herunterladen...")
    images = download_images(image_links)
    
    print("2. Waffen erkennen...")
    weapon_images = detect_weapons(images)
    
    print("3. Aggressive Posen analysieren...")
    aggressive_images = analyze_aggressive_poses(weapon_images)
    
    print(f"{len(aggressive_images)} aggressive Bilder gefunden.")
    return aggressive_images

# Bild-URLS 
image_links = ["http://example.com/image1.jpg", "http://example.com/image2.jpg"]
aggressive_images = main(image_links)

# Ergebnis ausgeben
print("Aggressive Bilder:", aggressive_images)

# Bereingng des Datensatzes


### Datenssatzverarbeitung 
Laden, Anzeigen und Sortieren von Bildern

In [ ]:
dataset_path = "path/to/your/images"
for file in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, file)
    img = cv2.imread(img_path)
    cv2.imshow("Image", img)
    key = cv2.waitKey(0)  # Sortieren mit Tasten, z.B. 'd' für delete
    if key == ord('d'):
        os.remove(img_path)
    cv2.destroyAllWindows()

### Waffenerkennung

#### PAPER

* Liu, Z., Xue, Z., Blum, R.S. et al. Concealed weapon detection and visualization in a synthesized image. Pattern Anal Applic 8, 375–389 (2006). https://doi.org/10.1007/s10044-005-0020-8
    - Bilder von heterogenen Bildsensoren liefern zusätzliche Informationen über die Szene. Das visuelle Bild identifiziert Personen, das Infrarot- oder Millimeterwellenbild erkennt verdächtige Bereiche mit Waffen. 
    - Die Informationen aus Multisensor-Bildern werden normalerweise mit der Technik der Multiresolution Pixel-Level Image Fusion integriert. Die Leistung von Multiresolution-Bildfusionsalgorithmen ist nicht immer zufriedenstellend, wenn die Bilder stark voneinander abweichen. In dieser Studie wird eine neue Strategie vorgeschlagen, die aus zwei Schritten besteht. Im ersten Schritt wird ein unüberwachtes Fuzzy-k-Means-Clustering verwendet.
    - Im zweiten Schritt wird der erkannte Bereich in das visuelle Bild integriert, wobei eine Mosaiktechnik mit mehreren Auflösungen verwendet wird. So wird das synthetisierte Bild verbessert, während der Bereich der verborgenen Waffe hervorgehoben wird. Die experimentellen Ergebnisse zeigen die Wirksamkeit des Ansatzes.
* 

* Open_Source_Modell: YOLO
    - https://docs.ultralytics.com/de/models/yolov8/
    - 